In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import gensim
from gensim import corpora,models
import pickle as pk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
import pickle
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from glove import Corpus,glove
import gensim
from gensim import corpora, models
stemmer = SnowballStemmer('english')
news = pd.read_csv('news_articles.csv')
news.head()
tokenizer = ToktokTokenizer()
stop_words = set(stopwords.words('english'))

ntopics = 8

In [2]:
news.head()

,Article_Id,Title,Author,Date,Content,URL
0,0,14 dead after bus falls into canal in Telangan...,Devyani Sultania,"August 22, 2016 12:34 IST",At least 14 people died and 17 others were inj...,http://www.ibtimes.co.in/14-dead-after-bus-fal...
1,1,Pratibha Tiwari molested on busy road Saath ...,Suparno Sarkar,"August 22, 2016 19:47 IST",TV actress Pratibha Tiwari who is best known ...,NaN
2,2,US South Korea begin joint military drill ami...,Namrata Tripathi,"August 22, 2016 18:10 IST",The United States and South Korea began a join...,http://www.ibtimes.co.in/us-south-korea-begin-...
3,3,Illegal construction in Bengaluru Will my hou...,S V Krishnamachari,"August 22, 2016 17:39 IST",The relentless drive by Bengaluru s Bangalore...,http://www.ibtimes.co.in/illegal-construction-...
4,4,Punjab Gau Rakshak Dal chief held for assaulti...,Pranshu Rathee,"August 22, 2016 17:34 IST",Punjab Gau Raksha Dal chief Satish Kumar and h...,http://www.ibtimes.co.in/punjab-gau-rakshak-da...


In [3]:
news.shape

(4831, 6)

In [4]:
articles_read = [2,7]
no_of_recommends = 5
passes = 5

In [5]:
news = news[['Article_Id','Title','Content']].dropna()
contents = news["Content"].tolist()

In [6]:
contents[0]

'At least 14 people died and 17 others were injured after a bus travelling from Hyderabad to Kakinada plunged into a canal from a bridge on the accident-prone stretch of the Hyderabad-Khammam highway in Telangana early Monday morning \r\nThe injured were admitted to the Government General Hospital for treatment \r\n\r\n\r\nSeven people died on the spot and the others succumbed to injuries while undergoing treatment at the hospital  The passengers belonged to the East and West Godavari districts of Andhra Pradesh \r\nThe bus  owned by private operator Yatra Genie  commenced its journey from Hyderabad at 11 30 p m  on Sunday  Khammam Superintendent of Police Shah Nawaz Khan was quoted by the Hindustan Times as saying \r\nThe accident happened around 2 30 a m  when the driver slammed the brakes to avoid a collision with another vehicle coming from the opposite direction on a bridge over Nagarjunsagar project left canal at Nayankangudem village in Khammam district  the daily reported  The 

In [7]:
def clean_tokenize(document):
    document = re.sub('[^\w_\s-]',' ',document)
    tokens  = nltk.word_tokenize(document)
    cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])   #stemming the tokenized corpus
    return cleaned_article

In [8]:
cleaned_articles = list(map(clean_tokenize,contents)) 

In [9]:
cleaned_articles[0]

'at least 14 peopl die and 17 other were injur after a bus travel from hyderabad to kakinada plung into a canal from a bridg on the accident-pron stretch of the hyderabad-khammam highway in telangana earli monday morn the injur were admit to the govern general hospit for treatment seven peopl die on the spot and the other succumb to injuri while undergo treatment at the hospit the passeng belong to the east and west godavari district of andhra pradesh the bus own by privat oper yatra geni commenc it journey from hyderabad at 11 30 p m on sunday khammam superintend of polic shah nawaz khan was quot by the hindustan time as say the accid happen around 2 30 a m when the driver slam the brake to avoid a collis with anoth vehicl come from the opposit direct on a bridg over nagarjunsagar project left canal at nayankangudem villag in khammam district the daili report the bus hit the parapet wall of the bridg and nose-div into the canal the driver of the bus was appar drive at high speed due t

In [10]:
user_articles = ' '.join(cleaned_articles[i] for i in articles_read) 

In [11]:
user_articles

'the unit state and south korea began a joint militari drill on monday which prompt threat from north korea the latter has late receiv strong critic worldwid for defi sanction from the unit nation secur council unsc by launch sever ballist missil such action have led to tighter sanction for north korea by the un north korea consid the joint militari drill as prepar for invas and has threaten a pre-empt nuclear strike if the u s and south korea continu the oper it had also conduct a nuclear test in januari which further isol it the ulchi freedom guardian exercis will continu till sept 2 and around 25 000 u s troop are expect to join it the us-l un command militari armistic commiss said that it had notifi the north korean armi that the joint militari drill between the two nation was not provoc from this moment the first-strik combin unit of the korean peopl s armi keep themselv fulli readi to mount a preemptiv retaliatori strike at all enemi attack group involv in ulji freedom guardian a

In [12]:
tfidf_matrix = TfidfVectorizer(stop_words='english',min_df=1)
article_tfidf_matrix = tfidf_matrix.fit_transform(cleaned_articles)
article_tfidf_matrix

<4831x36823 sparse matrix of type '<class 'numpy.float64'>'
	with 489462 stored elements in Compressed Sparse Row format>

In [13]:
user_article_tfidf_vector = tfidf_matrix.transform([user_articles])
user_article_tfidf_vector

<1x36823 sparse matrix of type '<class 'numpy.float64'>'
	with 195 stored elements in Compressed Sparse Row format>

In [14]:
article_tfidf_matrix.shape

(4831, 36823)

In [15]:
user_article_tfidf_vector

<1x36823 sparse matrix of type '<class 'numpy.float64'>'
	with 195 stored elements in Compressed Sparse Row format>

In [16]:
article_tfidf_matrix.shape,user_article_tfidf_vector.shape

((4831, 36823), (1, 36823))

In [17]:
articles_similarity_score = cosine_similarity(user_article_tfidf_vector,article_tfidf_matrix)

In [18]:
articles_similarity_score

array([[0.03049976, 0.0066453 , 0.79209201, ..., 0.01092048, 0.01228391,
        0.02579905]])

In [19]:
recommended_articles_ids = np.argsort(articles_similarity_score,axis=0)[::-1].flatten()

In [20]:
articles_similarity_score.shape

(1, 4831)

In [21]:
recommended_articles_ids

array([0, 0, 0, ..., 0, 0, 0])

In [22]:
final_recommended_articles_id = [article_id for article_id in recommended_articles_ids
                                if article_id not in articles_read][:no_of_recommends]

In [23]:
final_recommended_articles_id

[0, 0, 0, 0, 0]

In [24]:
print('Articles read by User :')
print (news.loc[news['Article_Id'].isin(articles_read)]['Title'])
print('Recommended Articles to User :')
print (news.loc[news['Article_Id'].isin(final_recommended_articles_id)]['Title'])

Articles read by User :
2    US  South Korea begin joint military drill ami...
7    Dialogue crucial in finding permanent solution...
Name: Title, dtype: object
Recommended Articles to User :
0    14 dead after bus falls into canal in Telangan...
Name: Title, dtype: object


# TOPIC-MODELLING (LDA)

In [25]:
#articlesreadbyusers
reads = []
reads_by_userA = [2,7]

reads_by_userB = [5,4]

reads_by_userC = [7,8]

reads_by_userD = [99,100]

reads = np.hstack([[reads_by_userA,reads_by_userB,reads_by_userC,reads_by_userD]])

In [26]:
news = pd.read_csv('news_articles.csv')

In [27]:
news.head()

,Article_Id,Title,Author,Date,Content,URL
0,0,14 dead after bus falls into canal in Telangan...,Devyani Sultania,"August 22, 2016 12:34 IST",At least 14 people died and 17 others were inj...,http://www.ibtimes.co.in/14-dead-after-bus-fal...
1,1,Pratibha Tiwari molested on busy road Saath ...,Suparno Sarkar,"August 22, 2016 19:47 IST",TV actress Pratibha Tiwari who is best known ...,NaN
2,2,US South Korea begin joint military drill ami...,Namrata Tripathi,"August 22, 2016 18:10 IST",The United States and South Korea began a join...,http://www.ibtimes.co.in/us-south-korea-begin-...
3,3,Illegal construction in Bengaluru Will my hou...,S V Krishnamachari,"August 22, 2016 17:39 IST",The relentless drive by Bengaluru s Bangalore...,http://www.ibtimes.co.in/illegal-construction-...
4,4,Punjab Gau Rakshak Dal chief held for assaulti...,Pranshu Rathee,"August 22, 2016 17:34 IST",Punjab Gau Raksha Dal chief Satish Kumar and h...,http://www.ibtimes.co.in/punjab-gau-rakshak-da...


In [28]:
news = news[['Article_Id','Title','Content']]

In [29]:
contents = news['Content'].tolist()

In [30]:
contents[0]

'At least 14 people died and 17 others were injured after a bus travelling from Hyderabad to Kakinada plunged into a canal from a bridge on the accident-prone stretch of the Hyderabad-Khammam highway in Telangana early Monday morning \r\nThe injured were admitted to the Government General Hospital for treatment \r\n\r\n\r\nSeven people died on the spot and the others succumbed to injuries while undergoing treatment at the hospital  The passengers belonged to the East and West Godavari districts of Andhra Pradesh \r\nThe bus  owned by private operator Yatra Genie  commenced its journey from Hyderabad at 11 30 p m  on Sunday  Khammam Superintendent of Police Shah Nawaz Khan was quoted by the Hindustan Times as saying \r\nThe accident happened around 2 30 a m  when the driver slammed the brakes to avoid a collision with another vehicle coming from the opposite direction on a bridge over Nagarjunsagar project left canal at Nayankangudem village in Khammam district  the daily reported  The 

In [31]:
# cleaning of articles
def clean_tokenize_filter(document):
    document = re.sub('[^\w_\s-]', ' ',document)       #remove punctuation marks and other symbols
    tokens = nltk.word_tokenize(document) #Tokenizing sentences
    filtered_sentence = [w for w in tokens if w.lower() not in stop_words] #removing stopwords
    cleaned_article = " ".join([i for i in filtered_sentence])
#     cleaned_article = ' '.join([stemmer.stem(item) for item in filtered_sentence])    #Stemming each token
    return cleaned_article

In [32]:
article_vocabulary = list(map(clean_tokenize_filter,contents))

In [33]:
article_vocabulary_matrix = tfidf_matrix.fit_transform(article_vocabulary)

In [34]:
article_vocabulary_matrix.shape

(4831, 45554)

In [35]:
tfidf_vocab = tfidf_matrix.vocabulary_

In [36]:
len(tfidf_vocab)

45554

In [37]:
#pk.dump(dictionary,open('dictionary_of_vocabulary.p','wb'))

In [38]:
lda = LatentDirichletAllocation(n_components=ntopics,max_iter=12,random_state=0)

In [39]:
article_mat  = lda.fit_transform(article_vocabulary_matrix) #converting tfidf matrix into documents and topics matrix [Docs,Topics] 

In [40]:
article_mat.shape

(4831, 8)

In [41]:
np.argmax(article_mat,axis=0).shape #returns the max value of topics in documents or returns which topic the document belongs to

(8,)

In [42]:
np.argmax(article_mat,axis=1) #returns the max value of topic from which most of the documents belong to 

array([7, 7, 7, ..., 5, 5, 5])

In [43]:
article_mat  

array([[0.01343567, 0.22498329, 0.01341198, ..., 0.01355663, 0.01344543,
        0.69435745],
       [0.14424052, 0.01576905, 0.01576702, ..., 0.01592627, 0.17126524,
        0.39491262],
       [0.0137364 , 0.01374001, 0.01373118, ..., 0.01375699, 0.01383342,
        0.9037057 ],
       ...,
       [0.01379233, 0.01379466, 0.01379205, ..., 0.90330092, 0.01383017,
        0.01388976],
       [0.01272748, 0.01280526, 0.01272566, ..., 0.91070791, 0.01273711,
        0.01284376],
       [0.01408462, 0.01408365, 0.01408162, ..., 0.59222305, 0.0140954 ,
        0.32326404]])

In [44]:
article_mat[1] #show how much a article belongs to the particular topic or can say the probability of the document to belong to a topic

array([0.14424052, 0.01576905, 0.01576702, 0.01576792, 0.22635135,
       0.01592627, 0.17126524, 0.39491262])

In [45]:
np.argmax(article_mat,axis=1)[1] #checking which topic does article one belongs to

7

In [46]:
cc = np.array(news["Title"]) #we take title of news from orginal raw data to recommend news accordingly

In [47]:
cc[1]

'Pratibha Tiwari molested on busy road   Saath Nibhana Saathiya  actress drags accused to police station'

In [48]:
cc[labels==6][:2]

NameError: name 'labels' is not defined

In [ ]:
cc[np.argmax(article_mat,axis=1)==6][:2]

In [ ]:
kmeans = KMeans(n_clusters=8,random_state=0,max_iter=4000)

In [ ]:
kmeans.fit(article_mat)

In [ ]:
labels = kmeans.labels_

In [ ]:
labels==5

In [ ]:
cc[labels==5][:2] #returns the matching labels where 5th topic is true in this example

In [ ]:
cc[2]

In [ ]:
cc[labels==5][:10]

In [ ]:
recommended_articles = []
for i in range(8):
    recommended_articles.append(cc[labels==i][:3])

In [ ]:
recommended_articles

# Glove

In [ ]:
f =  open(file='/home/linu/glove.6B.50d.txt',mode='r')    #pretrained file for glove with 4lakh words and 50 dimensions

In [ ]:
files = f.readlines()

In [ ]:
glove_model = [i.split(" ") for i in files]

In [ ]:
glove_model 

In [ ]:
glove_dictionary = {}    #empty dictionary
for i in range(len(files)) :     # here the length of file is 4lakh so the loop will run from 0 to 4lakh 
    row = files[i].split(' ')     # will iterate from 0 to 4lakh and split all the elements
    glove_dictionary[row[0]] = row[1:]  #here it creates a dictionary of row[0] 'keys' and 50dimension 'values'

In [ ]:
glove_words = list(glove_dictionary.keys())

In [ ]:
tfidf_words = list(tfidf_vocab.keys())

In [ ]:
final_dict = {}
for i in range(len(tfidf_words)):
    try:
        row = files[i].split(' ')
        final_dict[row[0]] = row[1:]
    else:
        final_dict[tfidf_words[i]] = [0]*50

In [ ]:
len(final_dict)

In [ ]:
len(tfidf_words)